# 9. hét / Prológus

A mai órán a következőkről lesz szó:
- Elméleti bevezetés a Deep Learningbe (~ 20 perc)
- Regresszió mélytanulás segítségével: California Housing Prices (~ 30 perc)
- Klasszifikáció mélytanulás segítségével: MNIST (~ 30 perc)

# 9. hét / I. Elméleti bevezetés a Deep Learningbe

## Ismétlés: Machine Learning feladatok megoldása

Jellemzően a gépi tanulás feladatai az alábbi lépésekből épülnek fel:
1. **Adatgyűjtés**, mérés:<span style="color:red"> Az adattudományok nem fognak működni adatok nélkül!</span> Fontos, hogy a mérések elvégzése előtt tegyünk becsléseket arra vonatkozóan, hogy mennyire sok adatra van szükségünk és mekkora paramétertérrel akarunk dolgozni! Minél több paramétert mérünk, annál több adatra van szükségünk! (A paramétertér dimenziójával exponenciálisan növekszik a szükséges adattér "térfogata".)

2. **Adatfeltérképezés**: alapvető - statisztikai eszközökkel történő - megismerkedés az adatbázisunkkal. Például ábrázoljuk az egyes attribútumok eloszlását, szórását, vagy kirajzoljuk a térbeli és időbeli folyamatokat.

3. ! **Adatok előkészítése**, data preprocessing: az adathalmaz előkészítése a modellek betanításának megfelelően. Ez maga a "kreatív agya" az adattudományoknak, amikor a nyers adatbázisból kiszűrjük a számunkra lényeges részeket. Az adat minőségi paramétereitől függően teljesen eltérő eszközöket igényel: például egy képet gyakran fekete-fehérre átállítunk, átméretezünk, megkeressük és kiemeljük rajta az éleket, hogy kizárólag a lényeges információtartalommal rendelkező adattal dolgozzon az algoritmusunk. Minél ügyesebben tudjuk elvégezni az adathalmaz előkészítését, annál több nehézségtől mentjük meg magunkat meg a tanítás során!

4. **Modellválasztás**: a tanító modell meghatározása. (*Megjegyzés: valójában a modellt ismernünk kell már a preprocessing előtt, elvégre a válaszott modellre fogjuk optimalizálni a nyers adathalmazt!*) Ez mindig azzal kezdődik, hogy definiáljuk a problémát (például klasszifikációt, vagy regressziót akarunk-e csinálni) és ennek megfelelően keresünk a szakirodalomban megfelelő megoldásokat. Ezek alapján összeállítjuk a modellünket.

5. **Modell illesztés**, tanítás: a választott modell betanítása.

6. **Kiértékelés**: a tanító folyamat kiértékelése megadott teljesítménymetrikák (például *precision*, *recall*, vagy *accuracy*) alapján. Érdemes ezek változását is ábrázolni a tanítás során.

7. **Modell paramétereinek finomhangolása**: az előfeldolgozás, vagy a választott modellünk módosítása a teljesítménymetrikák szerint; illetve a tanítás során fellépő rendellenességek és anomáliák feltérképezése. Gyakran előfordul, hogy ugyan működik is a modellünk, de esetleg nem elég pontos, nem elég megbízható, nem tud eléggé általánosítani, túltanult. Ilyenkor az első négy lépésen visszafelé érdemes elkezdeni végighaladni és onnan újrakezdve optimalizálni a modellt. 

8. **Alkalmazás**: az előre meghatározott célnak megfelelő modell telepítése.

## A perceptron modell

![perceptron](perceptron.png)

# 9. hét / II. Regresszió: California Housing Prices

Az alábbiakban meg fogunk ismerkedni a neurális hálók egy klasszikus "Hello World!" feladatával.

Adott egy adathalmazunk, amely a California állambeli házak árát tartalmazza 8 paraméter függvényében. Feladatunk az, hogy készítsünk egy olyan matematikai modellt, amely képes predikciókat tenni arra vonatkozóan, hogy az adatbázisba egy újonnan bekerülő ingatlant mennyi pénzbe kerülhet.

Az adatbázis elérhető az [alábbi linken](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset).

**Megoldás**:

Szükséges framework telepítése:

In [ ]:
%pip install tensorflow

### Szükséges importok

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import copy
import pandas as pd
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler

nyolc = 9
np.random.seed(nyolc)

### 1. Adatgyűjtés

Az adatgyűjtés problémáját a megfelelő szaktárgyak fedik le, mint például a Méréstechnika, Szenzor- és aktuátortechnika, Mikrovezérlők programozása, illetve egyes specializációkon ebben még részletesebben is elmélyülhetnek az érdeklődők.

Jelen kurzus nem fektet különösebb hangsúlyt arra, hogy a résztvevők a mérésadatgyűjtés és jelfeldolgozás eszközeiben jobban elmélyedjenek. Ennek elsődleges oka, hogy ezekről már külön-külön is egy teljes féléves anyagot össze lehetne állítani. A másik nagyon fontos dolog, hogy mérni (különösen akkora méretekben, mint ahogy azt az adattudományok igénylik! Emlékeztetőül: ha a vizsgált attribútumok száma $n$, akkor a szükséges adathalmaz mérete $2^n$-nel arányos!) roppant **költséges** folyamat! Gondoljunk csak arra, hogy a pontos mérésekhez mennyire sok szenzorra és mindeneklőtt üzemórára van szükségünk.

Ezen megfontolásokkal a tárgy keretein belül mindig feltételezzük, hogy **a megoldani kívánt feladat szempontjából alkalmas adatbázis áll rendelkezésre!**

In [ ]:
from sklearn.datasets import fetch_california_housing

california_housing = fetch_california_housing(as_frame=True)

### 2. Adatfeltérképezés

Jellemzően az adatfeltérképezés során rengeteg ábrát és statisztikai eszközt érdemes ráereszteni az adatbázisunkra. Mivel jelen esetben egy nagyon jól ismert adathalmazról van szó, ettől most eltekinthetünk. Részletes leírást az [alábbi linken](https://inria.github.io/scikit-learn-mooc/python_scripts/datasets_california_housing.html) kaphatunk.

Többek között azért fontos ez, hogy például ki tudjunk választani olyan attribútumokat, vagy változókat amelyek feltehetően teljesen irrelevánsak. Vagy éppen előre megjósolhassuk, hogy vajon mik lesznek a fontos paraméterek. 

In [ ]:
california_housing.frame

### 3. Data preprocessing

**3.1 Az adathalmaz felbontása**

Nagyon fontos lépés, hogy miután preprocesszáltuk a nyers adatot, utána osszuk fel három különböző részre. Ezek mind más funkciót fognak betölteni a *tanítás* során!
- **Training set**, vagy *tanító adathalmaz*: az adatnak az a része, amelyet ténylegesen fel fog dolgozni a neurális háló. Elsősorban ez az adathalmaz fogja definiálni a neurális háló főbb paramétereit. 

- **Validation set**, vagy *validációs adathalmaz*: a kiindulási adathalmaz azon része, amelyet ugyan tanítás közben folyamatosan lát a neurális háló, de az ebből szerzett információt csak a modell finomhangolására és hiperparaméteroptimalizálására használja. Például a validációs adathalmaz segítségével könnyen ellenőrizhetővé válik, hogy hány *epoch* után érdemes leállítani a tanítást, hogy elkerüljük az overfitting-et! (A validációs adathalmazt definiálni nem kötelező, de az esetek túlnyomó részében több, mint érdemes!)

- **Test set**, vagy *tesztelési adathalmaz*: azon adathalmaz, amelyet a tanítás végeztével arra használunk, hogy a neurális háló metrikáit kiértékeljük. Ezt az adathalmazt tanítás során semmiképp sem láthatja a neurális háló, ugyanis az meghamisítaná a modell teljesítményét!

Ökölszabályként érdemes megjegyezni, hogy jellemzően 60%-20%-20%, 70%-15%-15% arányokat szoktunk választani! Viszont fontos emellett, hogy ezen halmazoknak nem is az aránya, sokkal inkább a homogenitása ami mérvadó! Nagyon fontos, hogy mindhárom halmazban minél ideálisabban legyen reprezentálva a teljes adathalmaz, különben sok anomáliával szembesülhetünk! *(Például fontos, hogy egy rendezett adathalmazt ne egymás utáni részekre osszuk fel, hanem véletlenszerűen válasszuk ki az egyes elemeket! Ha nem így teszünk, akkor könnyen előfordulhat, hogy tanítás során például közel 100%-os accuracy értéket kapunk, a tesztelés során pedig ettől jelentősen kevesebb érték jelenik meg.)*

**3.2 Az adathalmaz standardizálása**

Szigorúan az adathalmaz felosztása után érdemes egyenként a tanító, validációs és tesztelési adathalmazainkat standardizálni. Ez azt jelenti, hogy a $\mathbf{X}$ adathalmazunk elemeit **normalizál**juk, azaz úgy transzformáljuk, hogy az $E$ **várható érték**e (jelen esetben: számtani közép, amit szokás még $\mu$-vel is jelölni) **zérus**, illetve a $\sigma$ **szórás**a **egységnyi** legyen! 
$\begin{equation} \hat{\mathbf{X}} = \frac{\mathbf{X}-E}{\sigma} \end{equation}$
Erre azért van szükség, hogy egyrészt ne kelljen feleslegesen nagy számokkal dolgoznia a neurális hálónak, másrészt pedig azért, hogy könnyebben tudjon általánosítani a modell.

*(Megjegyzés: miért nem mindegy, hogy az adathalmaz felbontása előtt, vagy után standardizáljuk az adatot? Melyik adathalmaznak és melyik aspektusa sérülne, ha fordítva járnánk el?)*

In [ ]:
dataset = california_housing.frame.values

# Tanító-validációs-tesztelő adathalmazok méretének definiálása
test_split = 
valid_split =

X = 
Y = 

# A megfelelő arányokban felosztjuk az egyes adathalmazokat
v_index = 
t_index = 

# Tesztelő adat: 90%-tól végig
X_test =
Y_test = 

# Validációs adat: 80%-90%-ig
X_valid =
Y_valid =

# Tanító adat: elejétől a 80%-ig
X =
Y = 
# Normalizáljuk az adatot (σ = 1, E = 0)


### 4. Modellválasztás

![sgd.jpg](sgd.jpg)

In [ ]:
# Callback függvények definiálása

# Modell felépítése

# Model tanításának hiperparaméterei


### 5. Modell illesztés

Miután definiáltuk az összes szükséges paramétert a modellünkben, kezdődhet maga a tanítási folyamat. Fontos megjegyezni, hogy ez egy masszívan számításigényes folyamat, ezért egy-egy tanítás könnyűszerrel igénybevehet több órát, de akár napokat is! Emiatt mindig érdemes egyszerűbb modelleket felépíteni, kevesebb réteggel és kevesebb neuronnal, a gyors betanulás érdekében. Ezen csak akkor érdemes javítani, ha a cél szempontjából jelentősen alulteljesít a modellünk.

### 6. Kiértékelés

In [ ]:
from sklearn.metrics import mean_squared_error
import math

# Betöltjük az elmentett modellt

# Predikálunk a teszt adatbázison

# Kiértékeljük a tesztelési adatbázison a modell hibáját
test_err = 

print("\nTeszt hiba: %f" % (test_err))
print(f"Ez az jelenti, hogy sqrt({test_err:0.3f}))={math.sqrt(test_err):0.3f}-et hibáz átlagosan a modell a teszt adatokon.")
print(f"Ez annyit jelent, hogy átlagosan {math.sqrt(test_err)*100000:0.0f}$-t téved a modellünk.")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
import seaborn as sns

plt.figure(figsize=(10,10))
fig = sns.regplot(x=Y_test, y=preds.reshape(-1));
# fig.set(xlim=(10,30),ylim=(10,30))
plt.xlabel("Ground truth [$ 100 000]")
plt.ylabel("Predictions [$ 100 000]")
plt.show()

### 7. A modell finomhangolása

Jelen esetben megállapíthatjuk, hogy a modell tanítása során nem jelentkezett semmilyen anomálisa, vagy rendellenesség. Nem állapítható meg túltanulás sem, ezért a kapott modellt **elfogadom**.

Ezen felül még érdemes figyelni, hogy mennyire gyorsan tud predikálni a modell és figyelembe venni, hogy vajon emellett képesek vagyunk-e valós idejű alkalmazást létrehozni. Ha nem, akkor célszerű egy egyszerűbb modellt választani.

### 8. Alkalmazás

Ha már nem tartunk igényt a modell további finomhangolására, akkor a `weights.keras` file-t elmentve készen állunk arra, hogy alkalmazásba ültessük a neurális hálónkat.

# 9. hét / III. Klasszifikáció: MNIST

Jelen példában ismételten egy "Hello World" típusú feladatot fogunk megvizsgálni, viszont jelen esetben már nem egy MLP (Multi Layer Perceptron) modellt fogunk segítségül hívni, hanem egy úgynevezett Konvolúciós Neurális Hálót, röviden CNN-t!

A feladat a következő: adott az MNIST adatbázis, amely rengeteg kézzel írott számjegyről tartalmaz 256x256-os felbontásban képeket. A feladatunk az, hogy készítsünk egy olyan modellt, amely képes egy újonnan beadott képről eldönteni, hogy azon milyen számjegy található!

Mielőtt mélyebbre áskálódnánk, fontos tisztázni, hogy miből is épül fel egy CNN! Tekintsük az alábbi példát: 
![cnn](2DCNN.png)

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Betöltjük az adatbázist - itt eleve szét van szedve tanító és teszt adathalamzra


In [ ]:
x_train =
x_test =

x_train = 
x_test = 
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

batch_size=128

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Embedding
from tensorflow.keras.optimizers import SGD


print(model.summary())

In [ ]:
# Callback
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

patience=10
early_stopping=EarlyStopping(patience=patience, verbose=1)
checkpointer=ModelCheckpoint(filepath='model.keras', save_best_only=True, verbose=1)

# tb = TensorBoard(log_dir='logs', histogram_freq=1, write_graph=1)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

network_history = model.fit(x_train, y_train, batch_size=128, epochs=30, verbose=1, validation_split=0.2, callbacks=[early_stopping, checkpointer, tb])

In [ ]:
from keras.models import load_model

model = load_model("mopdel.keras")
test_err = model.evaluate(x_test,y_test)
print("Teszt hiba:", test_err[0], "Teszt pontosság:", test_err[1])

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix,classification_report

y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred,1)
y_true = np.argmax(y_test,1)

print("test accuracy: %g" %(accuracy_score(y_true, y_pred)))
print("Precision", precision_score(y_true, y_pred, average="macro"))
print("Recall", recall_score(y_true, y_pred, average="macro"))
print("f1_score", f1_score(y_true, y_pred, average="macro"))
print("\nKonfúziós mátrix: ")
conf=confusion_matrix(y_true, y_pred)
print(conf)

In [ ]:
import seaborn as sns
ax = sns.heatmap(conf, annot=True, fmt='d', vmax=20, cmap='Blues') # a vmax paraméterrel állítjuk be, hogy milyen értéktartományban jelenítse meg az adatokat
ax.set(xlabel='Predicted Label',
       ylabel='True label');